# 프로젝트 3 : 패션 아이템 분류 딥러닝 신경망 설계

### 개요 : Fashion Mnist 데이터셋이용 패셔 아이템 인식하여 레이블 예측하기.
### 입력X와 레이블 y를 받아 학습 후, 새로운 X가 왔을 때 어떤 아이템인지 예측
### 레이어 3개 3층 신경망 모델

- 이용 모듈
- torch.nn : 인공신경망 모델 재료
- torch.optim : 최적화
- torch.nn.functional : nn모듈의 함수버젼

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

# CUDA 사용 여부 확인

- torch.cuda.is_available() : CUDA 용 PyTorch와 CUDA설치 시 True 반환

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

# 하이퍼 파라미터 설정

In [4]:
EPOCHS = 30
BATCH_SIZE = 64

# 데이터셋 불러오기

In [6]:
transform = transforms.Compose([
    transforms.ToTensor()  
])

In [8]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform
)

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

# 신경망 설계하기

- 입력'x'는 [배치크기, 색, 높이, 넓이]로 구성
- x.size() -> [64,1,28,28]로 표시 (64 = BATCH_SIZE, 하이퍼 파라미터)

- 입력 x : 28 x 28 x 1 = 784 


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256) # fully connected(Dense), 784개 입력받아 256개 출력
        self.fc2 = nn.Linear(256, 128) # 256개 입력받아 128개 출력
        self.fc3 = nn.Linear(128, 10)  # 128개 입력받아 10개 출력(패션 아이템 카테고리 개수)
        
    def forward(self, x):
        x = x.view(-1, 784)            # 입력을 받아 1차원 행렬로 만듦
        x = F.relu(self.fc1(x))        # nn.ReLU 클래스와 같은 기능
        x = F.relu(self.fc2(x))
        x = self.fc3(x)                # 출력 10개의 값
        return x

# 모델 선언하기

- to() 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할
- CPU 1개만 사용할 경우는 필요 없지만 , GPU를 사용하고자 하는 경우 'to("cuda")'로 지정하여 GPU로 보내야 함
- 지정하지 않을 경우 계속 CPU에 남아 있게 되며 GPU 이용한 빠른 학습이 안됨

- optim.SGD : 최적화 함수(SGD : 확률적 경사하강법)
- model.parameters() : 모델 내부 정보를 넘겨주는 함수
- lr(learning rate : 학습률) : 임의로 지정( 적당한 값, 처음에는 큰 값에서 점점 작아지게 설계하면 좋음)

In [10]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습모델 설계하기

- model.train() : 모델을 '학습'모드로 선언, 학습/평가 모드에 따라 동작이 다른 함수(Drop Out) 오동작 예방
- optimizer.zero_grad() : 기울기 초기화, 반복 때마다 기울기를 새로 계산하기 위해 호출

In [14]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader) :
        #학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE) # to()함수로 모델
                                                   #가중치에 사용한 같은 장치 이용(CPU or GPU)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)     # 예측값(output)과 레이블(target) 오차 구하기
                                                   # 교차 엔트로피
        loss.backward()                            #기울기 계산
        optimizer.step()                           #계산한 기울기를 모델에 맞춰 가중치 수정

# 딥 러닝 모델 성능 테스트하기

- epoch 끝날 때마다 테스트 셋으로 성능 평가
- 모델 평가 목표 시에는 최적화 X

- model.eval() : 평가 모드로 모델 전환
- torch.no_grad() : 평가에선 기울기 계산 안함
- output.max() : 가장 큰 값 & 인덱스 출력, 그 중 인덱스 [1] = 패션 아이템 번호 0~9
- pred.eq() : 값이 일치하면 1, 아니면 0
- view_as() : target 텐서를 ()안 인자의 모양대로 다시 정렬

In [15]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader :
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            # 모든 오차 더하기, 미니배치 평균이 아닌 합 ie. test_loss는 모든 테스트셋의 오차합
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 에측과 정답을 비교하여 일치할 경우 correct에 1을 더함
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            # sun90으로 배치에서 모델이 정답을 맞춘 개수를 구함
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# 코즈 돌려 모델 확인해보기

In [16]:
for epoch in range(1, EPOCHS + 1) :
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(
    epoch, test_loss, test_accuracy))

[1] Test Loss : 0.8413, Accuracy : 68.57%
[2] Test Loss : 0.6774, Accuracy : 76.44%
[3] Test Loss : 0.6232, Accuracy : 76.85%
[4] Test Loss : 0.5428, Accuracy : 80.87%
[5] Test Loss : 0.5451, Accuracy : 80.90%
[6] Test Loss : 0.4918, Accuracy : 82.53%
[7] Test Loss : 0.4780, Accuracy : 82.90%
[8] Test Loss : 0.4697, Accuracy : 83.55%
[9] Test Loss : 0.4860, Accuracy : 82.78%
[10] Test Loss : 0.4680, Accuracy : 83.10%
[11] Test Loss : 0.4542, Accuracy : 83.86%
[12] Test Loss : 0.4458, Accuracy : 84.25%
[13] Test Loss : 0.4516, Accuracy : 83.57%
[14] Test Loss : 0.4256, Accuracy : 85.04%
[15] Test Loss : 0.4270, Accuracy : 84.78%
[16] Test Loss : 0.4217, Accuracy : 84.99%
[17] Test Loss : 0.4258, Accuracy : 84.97%
[18] Test Loss : 0.4184, Accuracy : 85.25%
[19] Test Loss : 0.4752, Accuracy : 82.26%
[20] Test Loss : 0.4094, Accuracy : 85.29%
[21] Test Loss : 0.4002, Accuracy : 85.63%
[22] Test Loss : 0.4029, Accuracy : 85.91%
[23] Test Loss : 0.4047, Accuracy : 85.61%
[24] Test Loss : 0.3